# 量的データと質的データの可視化

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shinchu/dataviz-notebooks/blob/main/week_4/intro-to-visualizing-quantitative-and-qualitative-data.ipynb)

今回は、文化庁の[メディア芸術データベース・ラボ（MADB Lab）](https://mediag.bunka.go.jp/madb_lab/)で公開されている四大少年誌（週刊少年サンデー、週刊少年ジャンプ、週刊少年チャンピオン、週刊少年マガジン）のデータを使って、量的データと質的データの可視化を練習します。

まず、「四大少年誌それぞれの掲載作品のジャンルと著者にはどのような特徴があるのか？」という大きな問いを立て、可視化手法を学びながらデータを見て、具体的な問いを決めていきましょう。

[マンガと学ぶデータビジュアライゼーション](https://kakeami.github.io/viz-madb/index.html)の内容を全面的に参考にしています。

ここではPlotlyというライブラリを使います。MatplotlibとAltairの中間のような書き方ができるライブラリですが、Altairほど明示的にデータと視覚記号、データ変数と視覚変数の対応関係をとりません。

書き方は難しくないので、以下のコードを読んで把握しておきましょう。

今日の演習では、以下の中から好きな図を3つ選んで、Altairで再現してもらいます。

## ライブラリの読み込み

In [ ]:
# ライブラリのインストール。初回のみ実行。
!pip install plotly

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.figure_factory as ff

In [ ]:
import itertools
import warnings
warnings.filterwarnings('ignore')

## 準備関数

In [ ]:
# weekdayを曜日に変換
WD2STR = {
    0: 'Mon.',
    1: 'Tue.',
    2: 'Wed.',
    3: 'Thu.',
    4: 'Fri.',
    5: 'Sat.',
    6: 'Sun.',}

In [ ]:
def show_fig(fig):
    """Jupyter Bookでも表示可能なようRendererを指定"""
    fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
    fig.show()

In [ ]:
def add_years_to_df(df, unit_years=10):
    """unit_years単位で区切ったyears列を追加"""
    df_new = df.copy()
    df_new['years'] = \
        pd.to_datetime(df['datePublished']).dt.year \
        // unit_years * unit_years
    df_new['years'] = df_new['years'].astype(str)
    return df_new

In [ ]:
def add_weekday_to_df(df):
    """曜日情報をdfに追加"""
    df_new = df.copy()
    df_new['weekday'] = \
        pd.to_datetime(df_new['datePublished']).dt.weekday
    df_new['weekday_str'] = df_new['weekday'].apply(
        lambda x: WD2STR[x])
    return df_new

In [ ]:
def add_mcid_to_df(df):
    """mcnameのindexをdfに追加"""
    df_new = df.copy()
    mcname2mcid = {
        x: i for i, x in enumerate(df['mcname'].unique())}
    df_new['mcid'] = df_new['mcname'].apply(
        lambda x: mcname2mcid[x])
    return df_new

In [ ]:
def resample_df_by_cname_and_years(df):
    """cnameとyearsのすべての組み合わせが存在するように0埋め
    この処理を実施しないと作図時にX軸方向の順序が変わってしまう"""
    df_new = df.copy()
    yearss = df['years'].unique()
    cnames = df['cname'].unique()
    for cname, years in itertools.product(cnames, yearss):
        df_tmp = df_new[
            (df_new['cname'] == cname)&\
            (df_new['years'] == years)]
        if df_tmp.shape[0] == 0:
            s = pd.Series(
                {'cname': cname,
                 'years': years,
                 'weeks': 0,},
                index=df_tmp.columns)
            df_new = df_new.append(
                s, ignore_index=True)
    return df_new

In [ ]:
def resample_df_by_creator_and_years(df):
    """creatorとyearsのすべての組み合わせが存在するように0埋め
    この処理を実施しないと作図時にX軸方向の順序が変わってしまう"""
    df_new = df.copy()
    yearss = df['years'].unique()
    creators = df['creator'].unique()
    for creator, years in itertools.product(creators, yearss):
        df_tmp = df_new[
            (df_new['creator'] == creator)&\
            (df_new['years'] == years)]
        if df_tmp.shape[0] == 0:
            s = pd.Series(
                {'creator': creator,
                 'years': years,
                 'weeks': 0,},
                index=df_tmp.columns)
            df_new = df_new.append(
                s, ignore_index=True)
    return df_new

## データの用意

四大少年誌の`1970-07-27`から`2017-07-06`までの全ての掲載作品のデータを使います。

すでに前処理がされているデータがあるので、それを使います。

In [ ]:
file = "https://raw.githubusercontent.com/shinchu/dataviz-notebooks/main/data/week_4/episodes.csv"

In [ ]:
df = pd.read_csv(file)

In [ ]:
df.shape

各週の掲載作品を一行ずつ格納しているため、合計で約18万行程度になります。

In [ ]:
df.columns

- `mcname`: 雑誌名（**M**gazine **C**ollection **NAME**）
- `miid`：雑誌巻号ID（**M**agazine **I**tem **ID**）
- `miname`: 雑誌巻号名（**M**agazine **I**tem **NAME**）
- `cid`: マンガ作品ID（**C**omic **ID**）
- `cname`: マンガ作品名（**C**omic **NAME**）
- `epname`: 各話タイトル（**EP**isode **NAME**）
- `creator`: 作者名
- `pageStart`: 開始ページ
- `pageEnd`: 終了ページ
- `numberOfPages`: 雑誌の合計ページ数
- `datePublished`: 雑誌の発行日
- `price`: 雑誌の価格
- `publisher`: 雑誌の出版社
- `editor`: 雑誌の編集者（編集長）
- `pages`: 各話のページ数（`pageEnd` - `pageStart` + 1）
- `pageEndMax`: 雑誌に掲載されているマンガ作品のうち，`pageEnd`の最大値
- `pageStartPosition`: 各話の`pageStart`の相対的な位置（`pageStart` / `pageEndMax`）

In [ ]:
df.head()

In [ ]:
df.describe()

欠損値を確認してみます。

特に`epname`と`publisher`の欠測が多いことがわかります．

In [ ]:
df.isna().sum().reset_index()

## 量を見る

### 棒グラフ

#### 作品別の掲載週数（上位20作品）

In [ ]:
# 作品ごとの週数を数える
df_plot = df.value_counts("cname").reset_index(name="weeks").head(20)

In [ ]:
fig = px.bar(df_plot, x='cname', y='weeks', 
             title='作品別の掲載週数')
fig.update_xaxes(title='作品名')
fig.update_yaxes(title='掲載週数')
show_fig(fig)

#### 作品別・年代別の掲載週数（上位20作品）

In [ ]:
# dfに10年区切りの年代情報を追加
df = add_years_to_df(df)

In [ ]:
# プロット用に集計
df_plot = df.groupby('cname')['years'].value_counts().\
    reset_index(name='weeks')
# 連載週数上位10作品を抽出
cnames = list(df.value_counts('cname').head(20).index)
df_plot = df_plot[df_plot['cname'].isin(cnames)].\
    reset_index(drop=True)
# cname，yearsでアップサンプリング
df_plot = resample_df_by_cname_and_years(df_plot)

In [ ]:
# 合計連載週数で降順ソート
df_plot['order'] = df_plot['cname'].apply(
    lambda x: cnames.index(x))
df_plot = df_plot.sort_values(
    ['order', 'years'], ignore_index=True)

In [ ]:
# 作図
fig = px.bar(
    df_plot, x='cname', y='weeks', color='years',
    color_discrete_sequence= px.colors.diverging.Portland,
    barmode='group', 
    title='作品別・年代別の合計掲載週数（集合棒グラフ）')
fig.update_xaxes(title='作品名')
fig.update_yaxes(title='合計連載週数')
show_fig(fig)

In [ ]:
# 作図
fig = px.bar(
    df_plot, x='cname', y='weeks', color='years',
    color_discrete_sequence= px.colors.diverging.Portland,
    barmode='stack', 
    title='作品別・年代別の合計連載週数（積上げ棒グラフ）')
fig.update_xaxes(title='作品名')
fig.update_yaxes(title='合計連載週数')
show_fig(fig)

### ヒートマップ

#### 作品別・年代別の掲載週数（上位20作品）

In [ ]:
# 1年単位で区切ったyearsを追加
df = add_years_to_df(df, 1)

In [ ]:
# プロット用に集計
df_plot = \
    df.groupby('cname')['years'].value_counts().\
    reset_index(name='weeks')
# 連載週刊上位10作品を抽出
cnames = list(df.value_counts('cname').head(20).index)
df_plot = df_plot[df_plot['cname'].isin(cnames)].\
    reset_index(drop=True)
# 作図用に空白期間を0埋め
df_plot = \
    resample_df_by_cname_and_years(df_plot)

In [ ]:
# プロット用に集計
df_plot = \
    df.groupby('cname')['years'].value_counts().\
    reset_index(name='weeks')
# 連載週刊上位10作品を抽出
cnames = list(df.value_counts('cname').head(20).index)
df_plot = df_plot[df_plot['cname'].isin(cnames)].\
    reset_index(drop=True)
# 作図用に空白期間を0埋め
df_plot = \
    resample_df_by_cname_and_years(df_plot)

In [ ]:
fig = px.density_heatmap(
    df_plot, x='years', y='cname', z='weeks',
    title='作品別・年代別の合計掲載週数', height=500)
fig.update_xaxes(title='掲載年')
fig.update_yaxes(title='作品名')
show_fig(fig)

#### 作家別・年代別の合計掲載週数（上位20名）

In [ ]:
# 10年単位で区切ったyearsを追加
df = add_years_to_df(df, 1)

In [ ]:
# プロット用に集計
df_plot = \
    df.groupby('creator')['years'].value_counts().\
    reset_index(name='weeks')
# 連載週刊上位10作品を抽出
creators = list(df.value_counts('creator').head(20).index)
df_plot = df_plot[df_plot['creator'].isin(creators)].\
    reset_index(drop=True)
# 作図用に空白期間を0埋め
df_plot = \
    resample_df_by_creator_and_years(df_plot)

In [ ]:
# 合計連載週数で降順ソート
df_plot['order'] = df_plot['creator'].apply(
    lambda x: creators.index(x))
df_plot = df_plot.sort_values(
    ['order', 'years'], ignore_index=True)

In [ ]:
fig = px.density_heatmap(
    df_plot, x='years', y='creator', z='weeks',
    title='作家別・年代別の合計掲載週数', height=500)
fig.update_xaxes(title='掲載年')
fig.update_yaxes(title='作家名')
show_fig(fig)

## 分布を見る

### ヒストグラム

In [ ]:
# 平均掲載位置を算出する際の最小連載数
MIN_WEEKS = 5

#### 掲載位置の分布

`MIN_WEEKS`以上連載したマンガ作品の平均掲載位置の分布を見てみます。

In [ ]:
df_plot = \
    df.groupby(['mcname', 'cname', 'creator'])['pageStartPosition']\
    .agg(['count', 'mean']).reset_index()
df_plot = df_plot[df_plot['count'] >= MIN_WEEKS]\
    .reset_index(drop=True)

In [ ]:
fig = px.histogram(
    df_plot, x='mean', title='作品ごとの掲載位置')
fig.update_xaxes(title='平均掲載位置')
fig.update_yaxes(title='作品数')
show_fig(fig)

In [ ]:
fig = px.histogram(
    df_plot, x='mean', cumulative=True,
    title='作品ごとの掲載位置')
fig.update_xaxes(title='平均掲載位置')
fig.update_yaxes(title='累積作品数')
show_fig(fig)

In [ ]:
df_plot.sort_values('mean').reset_index(drop=True).head(10)

#### 長期連載作品の掲載位置の分布

In [ ]:
df_tmp = \
    df_plot.sort_values(['count'], ascending=False, ignore_index=True)\
    .head(10)
df_tmp

In [ ]:
cnames = df_tmp.sort_values('mean')['cname'].values
for cname in cnames:
    df_c = df[df['cname']==cname].reset_index(drop=True)
    pos = df_c['pageStartPosition'].mean()
    n = df_c.shape[0]
    fig = px.histogram(
        df_c, x='pageStartPosition', nbins=20,
        title=f'{cname}の掲載位置（全{n}話，平均{pos:.3f}）')
    fig.update_xaxes(title='掲載位置')
    fig.update_yaxes(title='話数')
    show_fig(fig)

### 密度プロット

#### 長期連載作品の掲載位置の分布

In [ ]:
df_tmp = \
    df.groupby('cname')['pageStartPosition']\
    .agg(['count', 'mean']).reset_index()
df_tmp = \
    df_tmp.sort_values('count', ascending=False, ignore_index=True)\
    .head(10)
cname2position = df_tmp.groupby('cname')['mean'].first().to_dict()

In [ ]:
df_plot = df[df['cname'].isin(list(cname2position.keys()))]\
    .reset_index(drop=True)
df_plot['position'] = df_plot['cname'].apply(
    lambda x: cname2position[x])
df_plot = df_plot.sort_values('position', ignore_index=True)

In [ ]:
cnames = df_tmp.sort_values('mean')['cname'].values
data = [
    df[df['cname']==cname].reset_index(drop=True)\
    ['pageStartPosition'] for cname in cnames]

In [ ]:
fig = ff.create_distplot(
    data, cnames, show_hist=False,
    colors= px.colors.sequential.Plasma_r)
fig.update_xaxes(title='掲載位置')
fig.update_yaxes(title='確率密度')
fig.update_layout(
    hovermode='x unified', height=600,
    title_text='長期連載作品の掲載位置')
show_fig(fig)

### 箱ひげ図

#### 長期連載作品の掲載位置の分布

In [ ]:
df_tmp = \
    df.groupby('cname')['pageStartPosition']\
    .agg(['count', 'mean']).reset_index()
df_tmp = \
    df_tmp.sort_values('count', ascending=False, ignore_index=True)\
    .head(10)
cname2position = df_tmp.groupby('cname')['mean'].first().to_dict()

In [ ]:
df_plot = df[df['cname'].isin(list(cname2position.keys()))]\
    .reset_index(drop=True)
df_plot['position'] = df_plot['cname'].apply(
    lambda x: cname2position[x])
df_plot = df_plot.sort_values('position', ignore_index=True)

In [ ]:
fig = px.box(
    df_plot, x='cname', y='pageStartPosition',
    title='長期連載作品の掲載位置')
fig.update_xaxes(title='作品名')
fig.update_yaxes(title='掲載位置')
show_fig(fig)

### バイオリンプロット

#### 長期連載作品の掲載位置の分布

In [ ]:
df_tmp = \
    df.groupby('cname')['pageStartPosition']\
    .agg(['count', 'mean']).reset_index()
df_tmp = \
    df_tmp.sort_values('count', ascending=False, ignore_index=True)\
    .head(10)
cname2position = df_tmp.groupby('cname')['mean'].first().to_dict()

In [ ]:
df_plot = df[df['cname'].isin(list(cname2position.keys()))]\
    .reset_index(drop=True)
df_plot['position'] = df_plot['cname'].apply(
    lambda x: cname2position[x])
df_plot = df_plot.sort_values('position', ignore_index=True)

In [ ]:
fig = px.violin(
    df_plot, x='cname', y='pageStartPosition', points=False,
    title='長期連載作品の掲載位置')
fig.update_traces(scalemode='count', meanline_visible=True, width=1)
fig.update_layout(violinmode='overlay', violingap=0)
fig.update_xaxes(title='作品名')
fig.update_yaxes(title='掲載位置')
show_fig(fig)

### リッジラインプロット

Plotlyでは直接リッジラインプロットを描く関数がないため、バイオリンプロットを応用します。Altairではリッジラインプロットを作れます。ぜひ試してみてください。

#### 長期連載作品の話数毎の掲載位置の分布

In [ ]:
df_tmp = \
    df.groupby('cname')['pageStartPosition']\
    .agg(['count', 'mean']).reset_index()
df_tmp = \
    df_tmp.sort_values('count', ascending=False, ignore_index=True)\
    .head(10)
cname2position = df_tmp.groupby('cname')['mean'].first().to_dict()

In [ ]:
df_plot = df[df['cname'].isin(list(cname2position.keys()))]\
    .reset_index(drop=True)
df_plot['position'] = df_plot['cname'].apply(
    lambda x: cname2position[x])
df_plot = df_plot.sort_values('position', ignore_index=True)

In [ ]:
# 話数の区切り
UNIT_EP = 100

In [ ]:
cnames = df_plot['cname'].unique()
for cname in cnames:
    df_c = df_plot[df_plot['cname']==cname].reset_index(drop=True)
    df_c['eprange'] = (df_c.index + 1) // UNIT_EP * UNIT_EP
    df_c = df_c.sort_values('eprange', ascending=False, ignore_index=False)
    df_c['eprange'] = df_c['eprange'].apply(
        lambda x: f'{x}話以降')
    fig = px.violin(
        df_c, y='eprange', x='pageStartPosition',
        title=f'{cname}の掲載位置', orientation='h',
        points=False)
    fig.update_traces(
        side='positive', scalemode='count', width=4)
    fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=False)
    fig.update_xaxes(title='掲載位置')
    fig.update_yaxes(title='話数')
    show_fig(fig)

## 比率を見る

### 円グラフ

#### 雑誌別の合計作品数

In [ ]:
df_plot = \
    df.groupby('mcname')['cname'].nunique().reset_index()
df_plot = \
    df_plot.sort_values(
    'cname', ascending=False, ignore_index=True)
fig = px.pie(
    df_plot, values='cname', names='mcname',
    color_discrete_sequence= px.colors.diverging.Portland,
    title='雑誌別の合計作品数')
show_fig(fig)

### 積み上げ棒グラフ

#### 雑誌別・年代別の合計作品数

In [ ]:
col_count = 'cname'

In [ ]:
# 10年単位で区切ったyearsを追加
df = add_years_to_df(df, 10)
# mcname, yearsで集計
df_plot = \
    df.groupby(['mcname', 'years'])[col_count].\
    nunique().reset_index()
# years単位で集計してdf_plotにカラムを追加
df_tmp = df_plot.groupby('years')[col_count].sum().reset_index(
    name='years_total')
df_plot = pd.merge(df_plot, df_tmp, how='left', on='years')
# years合計あたりの比率を計算
df_plot['ratio'] = df_plot[col_count] / df_plot['years_total']
df_plot['text'] = df_plot['ratio'].apply(
    lambda x: f'{x:.2}')

In [ ]:
fig = px.bar(
    df_plot, y='years', x='ratio', color='mcname', text='text',
    color_discrete_sequence= px.colors.diverging.Portland,
    barmode='stack', title='雑誌別・年代別の合計作品数')
fig.update_xaxes(title='期間')
fig.update_yaxes(title='比率')
show_fig(fig)

In [ ]:
fig = px.bar(
    df_plot, y='mcname', x='ratio', color='years', text='text',
    color_discrete_sequence= px.colors.diverging.Portland,
    barmode='group', title='雑誌別・年代別の合計作品数')
fig.update_xaxes(title='期間')
fig.update_yaxes(title='比率')
show_fig(fig)

### パラレルセットグラフ（サンキーチャート）

パラレルセットグラフは多変数の内訳を見るのに便利ですが、Altairでは作れないようです。

一方で、Plotlyでは簡単に作れます。

#### 雑誌別・年代別・曜日別の雑誌巻号数

In [ ]:
# 10年単位で区切ったyearsを追加
df_plot = df[~df['miname'].duplicated()].reset_index(drop=True)
df_plot = add_years_to_df(df_plot, 10)
df_plot = add_weekday_to_df(df_plot)
df_plot = add_mcid_to_df(df_plot)
df_plot = df_plot.sort_values(
    ['weekday', 'years', 'mcname'], ignore_index=True)

In [ ]:
fig = px.parallel_categories(
    df_plot, dimensions=['mcname', 'years', 'weekday_str'],
    color='mcid', 
    labels={
        'years': '年代', 'mcname': '雑誌名', 
        'weekday_str': '発売曜日'},
    title='雑誌別・年代別・曜日別の雑誌巻号数')
fig.update_coloraxes(showscale=False)
show_fig(fig)

## 変数の関係を見る

### 散布図

#### 雑誌別・作品別の平均掲載位置と連載週数

In [ ]:
df_plot = \
    df.groupby(['mcname', 'cname'])['pageStartPosition'].\
    agg(['count', 'mean']).reset_index()
df_plot.columns = ['mcname', 'cname', 'weeks', 'position']
df_plot = \
    df_plot[df_plot['weeks'] >= MIN_WEEKS].reset_index(drop=True)

In [ ]:
fig = px.scatter(
    df_plot, x='position', y='weeks', color='mcname', 
    opacity=0.7,
    hover_data=['cname'], 
    color_discrete_sequence= px.colors.diverging.Portland,
    title='雑誌別・作品別の平均掲載位置と連載週数')
fig.update_traces(
    marker={'size': 10, 'line_width':1})
fig.update_xaxes(title='平均掲載位置')
fig.update_yaxes(title='連載週数')
show_fig(fig)

### 二次元ヒストグラム

二次元のヒストグラム（ヒートマップの一種）は、散布図のドットが重複して確認しずらい時に使えます。

#### 作品別の平均掲載位置と連載週数

In [ ]:
df_plot = \
    df.groupby('cname')['pageStartPosition'].\
    agg(['count', 'mean']).reset_index()
df_plot.columns = ['cname', 'weeks', 'position']
df_plot = \
    df_plot[df_plot['weeks'] >= MIN_WEEKS].reset_index(drop=True)

In [ ]:
fig = px.density_heatmap(
    df_plot, x='position', y='weeks',
    title='作品別の平均掲載位置と掲載週数')
fig.update_xaxes(title='平均掲載位置')
fig.update_yaxes(title='掲載週数')
show_fig(fig)

In [ ]:
# 表示範囲を変更
fig.update_yaxes(range=[0, 200])
show_fig(fig)

### 並行座標プロット

並行座標プロットはパラレルセットグラフと似ていますが、量的変数を対象としているところが異なります。

パラレルセットグラフと同様、3つ以上の変数を扱う時に強みを発揮します。

#### 作品別の平均掲載位置と連載週数と平均ページ数

In [ ]:
df_plot = \
    df.groupby(['mcname', 'cname'])\
    [['pages', 'pageStartPosition']].\
    agg(['count', 'mean']).reset_index()
df_plot.columns = [
    'mcname', 'cname', 'weeks', 'pages',
    '_weeks', 'position']
df_plot = \
    df_plot[df_plot['weeks'] >= MIN_WEEKS].reset_index(drop=True)

In [ ]:
fig = px.parallel_coordinates(
    df_plot, dimensions=['position', 'pages', 'weeks'],
    color='position',
    labels={
        'position': '掲載位置', 'weeks': '掲載週数',
        'pages': 'ページ数'})
show_fig(fig)

### コレログラム

コレログラムは、複数の量的変数の相関係数を見るヒートマップです。

#### 作品別の平均掲載位置と掲載週数

In [ ]:
df_plot = \
    df.groupby(['cname'])\
    [['pages', 'pageStartPosition']].\
    agg(['count', 'mean']).reset_index()
df_plot.columns = [
    '作品名', '掲載週数', '平均ページ数',
    '_weeks', '平均掲載位置']
df_plot = \
    df_plot[df_plot['掲載週数'] >= MIN_WEEKS].\
    reset_index(drop=True)
df_plot = df_plot.drop(columns=['_weeks'])
df_corr = df_plot.corr().iloc[::-1]

In [ ]:
# 作図対象の相関係数
df_corr

In [ ]:
fig = ff.create_annotated_heatmap(
    df_corr.values,
    x=list(df_corr.index.values),
    y=list(df_corr.columns.values),
    annotation_text=df_corr.values,
    #colorscale='BlueRed_r'
)
show_fig(fig)

### 折れ線グラフ

#### 雑誌別の年毎のエピソード数の推移

In [ ]:
# 1年単位で区切ったyearsを追加
df = add_years_to_df(df, 1)

In [ ]:
df_plot = df.groupby(["years", "mcname"])["cname"].count().reset_index(name="count")

In [ ]:
fig = px.line(df_plot, x="years", y="count", color="mcname",
              labels={
                  "years": "年",
                  "count": "エピソード数",
                  "mcname": "雑誌名"
              },
              title="エピソード数の推移")
show_fig(fig)